# Ner_Train 파일 만들기

In [1]:
# 필요한 import 하기

import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
from utils.Preprocess import Preprocess
import matplotlib.pyplot as plt

In [2]:
from utils.Preprocess import Preprocess
p = Preprocess(word2index_dic=os.path.join('./train_tools/dict', 'chatbot_dict.bin'),
               userdic=os.path.join('./utils', 'train.tsv'))

def process():
    df_ner = pd.read_csv('ner_tag.csv', header=None) 
    df_ner.columns = ['word', 'tag', 'intent']
    
    df_intent = pd.read_csv('./models/intent/Intent_train_data.csv', header=None) 
    df_intent.columns = ['sentence', 'intent', 'intent_info']
    df_intent = df_intent.drop_duplicates()
    
    result = []
    sentence = []
    
    for intent in df_intent["sentence"]:
        result.append([])
        # First Line
        data = f'; {intent}'
        result[len(result) - 1].append(data)
        
        # Second Line
        idx = 0
        word = ""
        tag = ""
        for ner in df_ner['word']:
            if ner in intent:
                word = ner
                tag = df_ner['tag'][idx]
                break;
            idx += 1;
            
        data = f'${intent}'
        if word != "":
            data = data.replace(word, f'<{word}:{tag}>')
        result[len(result) - 1].append(data)
        
        # Third Line
        data = p.pos(intent)
        idx = 1
        for put in data:
            if put[0] == word:
                words = f"{idx}\t{put[0]}\t{put[1]}\tB_{tag}"
            else:
                words = f"{idx}\t{put[0]}\t{put[1]}\tO"
            
            idx += 1
            result[len(result) - 1].append(words)
    return result

result = process()

In [3]:
len(result)

10063

In [4]:
toText = ""

for data in result:
    for words in data:
        toText += words
        toText += "\n"
    toText += "\n"
    
toText

'; 빵종류?\n$<빵종류:MENU>?\n1\t빵종류\tNNG\tB_MENU\n2\t?\tSF\tO\n\n; 종류?\n$종류?\n1\t종류\tNNG\tO\n2\t?\tSF\tO\n\n; 음료종류?\n$<음료종류:MENU>?\n1\t음료종류\tNNG\tB_MENU\n2\t?\tSF\tO\n\n; 커피종류?\n$<커피종류:MENU>?\n1\t커피종류\tNNG\tB_MENU\n2\t?\tSF\tO\n\n; 식사종류?\n$<식사종류:MENU>?\n1\t식사종류\tNNG\tB_MENU\n2\t?\tSF\tO\n\n; 에이드종류?\n$<에이드종류:MENU>?\n1\t에이드\tNNG\tO\n2\t종류\tNNG\tO\n3\t?\tSF\tO\n\n; 사이드메뉴는 뭐 있어?\n$<사이드메뉴:MENU>는 뭐 있어?\n1\t사이드\tNNP\tO\n2\t메뉴\tNNP\tO\n3\t는\tJX\tO\n4\t뭐\tNP\tO\n5\t있\tVX\tO\n6\t어\tEF\tO\n7\t?\tSF\tO\n\n; 사이드 메뉴는 뭐 있어?\n$<사이드 메뉴:MENU>는 뭐 있어?\n1\t사이드\tNNP\tO\n2\t메뉴\tNNP\tO\n3\t는\tJX\tO\n4\t뭐\tNP\tO\n5\t있\tVX\tO\n6\t어\tEF\tO\n7\t?\tSF\tO\n\n; 빵종류는 뭐 있어?\n$<빵종류:MENU>는 뭐 있어?\n1\t빵종류\tNNG\tB_MENU\n2\t는\tJX\tO\n3\t뭐\tNP\tO\n4\t있\tVX\tO\n5\t어\tEF\tO\n6\t?\tSF\tO\n\n; 종류는 뭐 있어?\n$종류는 뭐 있어?\n1\t종류\tNNG\tO\n2\t는\tJX\tO\n3\t뭐\tNP\tO\n4\t있\tVX\tO\n5\t어\tEF\tO\n6\t?\tSF\tO\n\n; 음료종류는 뭐 있어?\n$<음료종류:MENU>는 뭐 있어?\n1\t음료종류\tNNG\tB_MENU\n2\t는\tJX\tO\n3\t뭐\tNP\tO\n4\t있\tVX\tO\n5\t어\tEF\tO\n6\t?\tSF\tO\n\n; 커피종류는 뭐 있어?\n

In [5]:
with open('./models/ner/ner_train.txt', 'w', encoding='utf-8') as f:
    f.write(toText)

In [6]:
# 학습 파일 불러오기
def read_file(file_name):
    sents = []
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ';' and lines[idx + 1][0] == '$':
                this_sent = []
            elif l[0] == '$' and lines[idx - 1][0] == ';':
                continue
            elif l[0] == '\n':
                sents.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))
    return sents

In [7]:
# ① 학습용 말뭉치 데이터를 불러옴
corpus = read_file(os.path.join('./models/ner', 'ner_train.txt'))

In [8]:
len(corpus)

10063

In [9]:
corpus[:20]

[[('1', '빵종류', 'NNG', 'B_MENU'), ('2', '?', 'SF', 'O')],
 [('1', '종류', 'NNG', 'O'), ('2', '?', 'SF', 'O')],
 [('1', '음료종류', 'NNG', 'B_MENU'), ('2', '?', 'SF', 'O')],
 [('1', '커피종류', 'NNG', 'B_MENU'), ('2', '?', 'SF', 'O')],
 [('1', '식사종류', 'NNG', 'B_MENU'), ('2', '?', 'SF', 'O')],
 [('1', '에이드', 'NNG', 'O'), ('2', '종류', 'NNG', 'O'), ('3', '?', 'SF', 'O')],
 [('1', '사이드', 'NNP', 'O'),
  ('2', '메뉴', 'NNP', 'O'),
  ('3', '는', 'JX', 'O'),
  ('4', '뭐', 'NP', 'O'),
  ('5', '있', 'VX', 'O'),
  ('6', '어', 'EF', 'O'),
  ('7', '?', 'SF', 'O')],
 [('1', '사이드', 'NNP', 'O'),
  ('2', '메뉴', 'NNP', 'O'),
  ('3', '는', 'JX', 'O'),
  ('4', '뭐', 'NP', 'O'),
  ('5', '있', 'VX', 'O'),
  ('6', '어', 'EF', 'O'),
  ('7', '?', 'SF', 'O')],
 [('1', '빵종류', 'NNG', 'B_MENU'),
  ('2', '는', 'JX', 'O'),
  ('3', '뭐', 'NP', 'O'),
  ('4', '있', 'VX', 'O'),
  ('5', '어', 'EF', 'O'),
  ('6', '?', 'SF', 'O')],
 [('1', '종류', 'NNG', 'O'),
  ('2', '는', 'JX', 'O'),
  ('3', '뭐', 'NP', 'O'),
  ('4', '있', 'VX', 'O'),
  ('5', '어', 'EF',

In [10]:
# ② 위에서 불러온 말뭉치 데이터에서 단어(w[1])와 BIO 태그(w[3])만 불러와 학습용 데이터셋 생성

sentences, tags = [], []
for t in corpus:
    tagged_sentence = []
    sentence, bio_tag = [], []
    for w in t:
        tagged_sentence.append((w[1], w[3]))
        sentence.append(w[1])
        bio_tag.append(w[3])
    
    sentences.append(sentence)
    tags.append(bio_tag)

In [11]:
sentences[:10]

[['빵종류', '?'],
 ['종류', '?'],
 ['음료종류', '?'],
 ['커피종류', '?'],
 ['식사종류', '?'],
 ['에이드', '종류', '?'],
 ['사이드', '메뉴', '는', '뭐', '있', '어', '?'],
 ['사이드', '메뉴', '는', '뭐', '있', '어', '?'],
 ['빵종류', '는', '뭐', '있', '어', '?'],
 ['종류', '는', '뭐', '있', '어', '?']]

In [12]:
tags[:10]

[['B_MENU', 'O'],
 ['O', 'O'],
 ['B_MENU', 'O'],
 ['B_MENU', 'O'],
 ['B_MENU', 'O'],
 ['O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['B_MENU', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O']]

In [13]:
print("샘플 크기 : \n", len(sentences))
print("0번 째 샘플 단어 시퀀스 : \n", sentences[0])
print("0번 째 샘플 bio 태그 : \n", tags[0])
print("샘플 단어 시퀀스 최대 길이 :", max(len(l) for l in sentences))
print("샘플 단어 시퀀스 평균 길이 :", (sum(map(len, sentences))/len(sentences)))

샘플 크기 : 
 10063
0번 째 샘플 단어 시퀀스 : 
 ['빵종류', '?']
0번 째 샘플 bio 태그 : 
 ['B_MENU', 'O']
샘플 단어 시퀀스 최대 길이 : 16
샘플 단어 시퀀스 평균 길이 : 6.18006558680314


In [14]:
# ③ 토크나이저 정의
# 단어 시퀀스의 경우 Preprocess 객체에서 생성하기 때문에 예제에서는 BIO 태그용 Tokenizer 객체 생성
tag_tokenizer = preprocessing.text.Tokenizer(lower=False) # 태그 정보는 lower=False 소문자로 변환하지 않는다.
tag_tokenizer.fit_on_texts(tags)  # 나중에 texts_to_sequences() 등을 수행하기 전에 꼭

In [15]:
# 생성된 사전 리스트를 이용해 단어와 태그 사전의 크기를 정의

In [16]:
# 우선 확인
p.word_index

{'OOV': 1,
 '하': 2,
 '주문': 3,
 '시': 4,
 '분': 5,
 '가능': 6,
 '예약': 7,
 '주': 8,
 '어요': 9,
 '고': 10,
 '싶': 11,
 '어': 12,
 '모레': 13,
 '내일': 14,
 '변경': 15,
 'ㄹ': 16,
 '취소': 17,
 '요': 18,
 '이': 19,
 '아': 20,
 '되': 21,
 '게': 22,
 'ㄴ': 23,
 '나요': 24,
 '있': 25,
 '수': 26,
 '다음': 27,
 '어떻': 28,
 'ㄴ가요': 29,
 '에': 30,
 '금주': 31,
 '로': 32,
 '는': 33,
 '을': 34,
 '나': 35,
 'ㅂ니다': 36,
 '가요': 37,
 '시간': 38,
 '으로': 39,
 '오늘': 40,
 '은데': 41,
 '다': 42,
 '가': 43,
 '덮밥': 44,
 '금일': 45,
 '밥': 46,
 '음식': 47,
 'ㄹ까요': 48,
 '문': 49,
 '아야': 50,
 '습니다': 51,
 '바로': 52,
 '4': 53,
 '었': 54,
 '은': 55,
 '1': 56,
 'ㄹ게요': 57,
 '겠': 58,
 '도': 59,
 '삼': 60,
 '국수': 61,
 '진행': 62,
 '콜': 63,
 '빵': 64,
 '거': 65,
 '드리': 66,
 '메뉴': 67,
 '의': 68,
 'ㄹ께': 69,
 '15': 70,
 '20': 71,
 '17': 72,
 '지': 73,
 '받': 74,
 '18': 75,
 '라면': 76,
 '았': 77,
 '11': 78,
 '지금': 79,
 '달걀': 80,
 '는데': 81,
 '5': 82,
 '면': 83,
 '날짜': 84,
 '톡': 85,
 '12': 86,
 '담': 87,
 '를': 88,
 '8': 89,
 '16': 90,
 '짜': 91,
 '19': 92,
 '14': 93,
 '보': 94,
 '당일': 95,
 '아요'

In [17]:
tag_tokenizer.word_index

{'O': 1,
 'NNG': 2,
 'B_PAY': 3,
 'B_MENU': 4,
 'B_FACILITY': 5,
 'B_RECOMMEND': 6,
 'B_FOOD': 7,
 'B_ORIGIN': 8,
 'B_POINT': 9,
 '브레드': 10,
 '케이크': 11,
 '플랫치노': 12,
 '라떼': 13,
 '마카롱': 14,
 '콜드브루': 15,
 '1인빙수': 16,
 '칩': 17,
 '스퀘어': 18,
 '스틱케익': 19,
 '샐러드': 20,
 '와플': 21,
 '비엔나': 22,
 '크림': 23,
 '스콘': 24,
 '샌드위치': 25,
 '번': 26,
 '모카': 27,
 '쿠키': 28,
 '위드샷': 29,
 '흑당': 30,
 '니트로': 31,
 '아메리카노': 32,
 '크림넛': 33,
 '화이트비엔나': 34,
 '구운주먹밥': 35,
 '크런치': 36,
 '밀크티': 37,
 '콤부차': 38,
 '과카몰리': 39,
 '오렌지': 40,
 '피자': 41}

In [18]:
# 단어사전 및 태그 사전 크기
vocab_size = len(p.word_index) + 1
tag_size = len(tag_tokenizer.word_index) + 1
print("BIO 태그 사전 크기 :", tag_size)
print("단어 사전 크기 :", vocab_size)

BIO 태그 사전 크기 : 42
단어 사전 크기 : 17752


In [19]:
# 학습용 단어 시퀀스 생성
x_train = [p.get_wordidx_sequence(sent) for sent in sentences]
y_train = tag_tokenizer.texts_to_sequences(tags)

index_to_ner = tag_tokenizer.index_word
index_to_ner[0] = 'PAD'

In [20]:
len(x_train)

10063

In [21]:
x_train[:10]

[[1, 1],
 [8941, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 8941, 1],
 [7837, 67, 33, 552, 25, 12, 1],
 [7837, 67, 33, 552, 25, 12, 1],
 [1, 33, 552, 25, 12, 1],
 [8941, 33, 552, 25, 12, 1]]

In [22]:
y_train[:10]

[[4, 1],
 [1, 1],
 [4, 1],
 [4, 1],
 [4, 1],
 [1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1],
 [4, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1]]

In [23]:
index_to_ner   # 0: 'PAD'  가 추가되었다

{1: 'O',
 2: 'NNG',
 3: 'B_PAY',
 4: 'B_MENU',
 5: 'B_FACILITY',
 6: 'B_RECOMMEND',
 7: 'B_FOOD',
 8: 'B_ORIGIN',
 9: 'B_POINT',
 10: '브레드',
 11: '케이크',
 12: '플랫치노',
 13: '라떼',
 14: '마카롱',
 15: '콜드브루',
 16: '1인빙수',
 17: '칩',
 18: '스퀘어',
 19: '스틱케익',
 20: '샐러드',
 21: '와플',
 22: '비엔나',
 23: '크림',
 24: '스콘',
 25: '샌드위치',
 26: '번',
 27: '모카',
 28: '쿠키',
 29: '위드샷',
 30: '흑당',
 31: '니트로',
 32: '아메리카노',
 33: '크림넛',
 34: '화이트비엔나',
 35: '구운주먹밥',
 36: '크런치',
 37: '밀크티',
 38: '콤부차',
 39: '과카몰리',
 40: '오렌지',
 41: '피자',
 0: 'PAD'}

In [24]:
# ⑤ 시퀀스 패딩 처리
max_len = 40   # 시퀀스 벡터 크기는 위에서 계산해본 평균 길이보다는 넉넉하게 40으로 설정
x_train = preprocessing.sequence.pad_sequences(x_train, padding='post', maxlen=max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding='post', maxlen=max_len)


In [25]:
x_train

array([[   1,    1,    0, ...,    0,    0,    0],
       [8941,    1,    0, ...,    0,    0,    0],
       [   1,    1,    0, ...,    0,    0,    0],
       ...,
       [ 415,    3,    2, ...,    0,    0,    0],
       [   1,    3,    2, ...,    0,    0,    0],
       [   1,    3,    2, ...,    0,    0,    0]])

In [26]:
y_train

array([[4, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [4, 1, 0, ..., 0, 0, 0],
       ...,
       [2, 1, 1, ..., 0, 0, 0],
       [2, 1, 1, ..., 0, 0, 0],
       [2, 1, 1, ..., 0, 0, 0]])

In [27]:
x_train.shape, y_train.shape

((10063, 40), (10063, 40))

In [28]:
# ⑥ 학습 데이터와 테스트 데이터를 8:2의 비율로 분리
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train,
                                                    test_size=.2,
                                                    random_state=12)


In [29]:
len(x_train), len(x_test), len(y_train), len(y_test)

(8050, 2013, 8050, 2013)

In [30]:
tag_size

42

In [31]:
# 출력 데이터를 one-hot encoding
# ★ 한번만 실행할것!
y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)  # tag_size. BIO 태그 사전 크기, 
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)    # 현재 tag_size 는 10 이다

In [32]:
y_train.shape, y_test.shape

((8050, 40, 42), (2013, 40, 42))

In [33]:
y_train

array([[[0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0.

In [34]:
print("학습 샘플 시퀀스 형상 : ", x_train.shape)
print("학습 샘플 레이블 형상 : ", y_train.shape)
print("테스트 샘플 시퀀스 형상 : ", x_test.shape)
print("테스트 샘플 레이블 형상 : ", y_test.shape)

학습 샘플 시퀀스 형상 :  (8050, 40)
학습 샘플 레이블 형상 :  (8050, 40, 42)
테스트 샘플 시퀀스 형상 :  (2013, 40)
테스트 샘플 레이블 형상 :  (2013, 40, 42)


In [35]:
# ⑦ 모델 정의 (Bi-LSTM)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=30, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])

In [36]:
model_file = os.path.join(r'./models/ner', 'ner_model.h5')
model_file

'./models/ner\\ner_model.h5'

In [37]:
# 학습하기
model.fit(x_train, y_train, batch_size=128, epochs=10)

Epoch 1/10
63/63 [==============================] - 30s 345ms/step - loss: 0.0886 - accuracy: 0.9005
Epoch 2/10
63/63 [==============================] - 20s 317ms/step - loss: 0.0271 - accuracy: 0.9497
Epoch 3/10
63/63 [==============================] - 20s 313ms/step - loss: 0.0166 - accuracy: 0.9707
Epoch 4/10
63/63 [==============================] - 20s 324ms/step - loss: 0.0129 - accuracy: 0.9770
Epoch 5/10
63/63 [==============================] - 20s 319ms/step - loss: 0.0115 - accuracy: 0.9791
Epoch 6/10
63/63 [==============================] - 21s 327ms/step - loss: 0.0106 - accuracy: 0.9806
Epoch 7/10
63/63 [==============================] - 21s 330ms/step - loss: 0.0101 - accuracy: 0.9812
Epoch 8/10
63/63 [==============================] - 22s 354ms/step - loss: 0.0099 - accuracy: 0.9819
Epoch 9/10
63/63 [==============================] - 22s 351ms/step - loss: 0.0097 - accuracy: 0.9823
Epoch 10/10
63/63 [==============================] - 22s 344ms/step - loss: 0.0093 - accura

In [38]:
print("평가 결과 : ", model.evaluate(x_test, y_test)[1])
model.save(os.path.join('./models/ner', 'ner_model.h5'))  # 학습 완료된 모델 저장

63/63 [==============================] - 2s 22ms/step - loss: 0.0096 - accuracy: 0.9829
평가 결과 :  0.9828915596008301


In [39]:
# F1 Score 계산

In [40]:
# 시퀀스를 NER 태그로 변환
def sequences_to_tag(sequences):  # 예측값을 index_to_ner를 사용하여 태깅 정보로 변경하는 함수.
    result = []
    for sequence in sequences:  # 전체 시퀀스로부터 시퀀스를 하나씩 꺼낸다.
        temp = []
        for pred in sequence:  # 시퀀스로부터 예측값을 하나씩 꺼낸다.
            pred_index = np.argmax(pred)  # 예를 들어 [0, 0, 1, 0 ,0]라면 1의 인덱스인 2를 리턴한다.
            temp.append(index_to_ner[pred_index].replace("PAD", "O"))  # 'PAD'는 'O'로 변경
        result.append(temp)
    return result

In [41]:
from seqeval.metrics import f1_score, classification_report

In [42]:
# 테스트 데이터셋의 NER 예측
# F1 score 를 게산하기 위해 모델의 predict() 함수를 통해 테스트용 데이터 셋의 결과를 예측합니다.
# 해당 함수의 입력은 시퀀스 번호로 인코딩 된 테스트 단어 시퀀스 (ndarray 배열)을 사용
# 해당 함수의 결과는 예측된 NER 태그 정보가 담긴 ndarray 배열을 리턴
y_predicted = model.predict(x_test)

In [43]:
pred_tags = sequences_to_tag(y_predicted) # 예측된 NER
test_tags = sequences_to_tag(y_test)    # 실제 NER

In [44]:
# F1 평가 결과
# seqeval.metrics 모듈의 classification+_report 함수를 통해 NER 태그 별로 계산된 precision, recall 그리고 f1_score 값을 출력

print(classification_report(test_tags, pred_tags))

# f1_score() 함수 만으로 f1 score 값만 출력 가능
print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))

C:\ProgramData\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_MENU seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\ProgramData\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_FOOD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\ProgramData\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: NNG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\ProgramData\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: 케이크 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\ProgramData\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_PAY seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\ProgramData\Anaconda3\lib\site-packages\seqeval\metrics\seq

              precision    recall  f1-score   support

          NG       0.10      0.91      0.18       268
   _FACILITY       0.89      0.97      0.93       122
       _FOOD       0.50      0.02      0.03        62
       _MENU       0.97      0.97      0.97       209
     _ORIGIN       0.86      0.67      0.75        36
        _PAY       0.93      1.00      0.96       213
      _POINT       0.80      0.73      0.76        11
  _RECOMMEND       0.95      1.00      0.98        82
           당       0.00      0.00      0.00         3
         드브루       0.00      0.00      0.00         5
          드샷       0.00      0.00      0.00         1
         드위치       0.00      0.00      0.00         3
           떼       0.00      0.00      0.00         4
         랫치노       0.00      0.00      0.00         6
          러드       0.00      0.00      0.00         2
          런치       1.00      1.00      1.00         2
          레드       0.00      0.00      0.00        10
          렌지       0.00    